In [ ]:
!pip install -e ".[dev]"

In [2]:
import os
import re
import logging
from dotenv import load_dotenv
from dataclasses import dataclass, asdict
from typing import Optional


_LOGGER = logging.getLogger(__name__)

load_dotenv()
_LOGGER.info("--- envs've been successfully loaded ---")

@dataclass
class GigaSettings:
    # --- ПАРАМЕТРЫ GIGACHAT API ---
    base_url: str
    verify_ssl_certs: bool=False
    main_model: str="GigaChat" # Изменил на Pro, так как Max может быть недоступен
    temperature: float=0.0

# Определяем URL. По умолчанию используем публичный API, который работает везде.
# Если нужен IFT контур, задайте переменную окружения GIGACHAT_API_URL
_api_url = os.getenv("GIGACHAT_API_URL", "https://gigachat.devices.sberbank.ru/api/v1")
# Убираем дублирование /v1 если оно уже есть в переменной окружения
if not _api_url.endswith("/v1") and "/api" not in _api_url:
     _api_url = f"{_api_url}/v1"

giga_settings = GigaSettings(
    _api_url,
    False
)

_LOGGER.info(f"--- GIGA CONFIG: {asdict(giga_settings)} ---")
print(f"--- GIGA CONFIG: {asdict(giga_settings)} ---")

INFO:__main__:--- envs've been successfully loaded ---
INFO:__main__:--- GIGA CONFIG: {'base_url': 'https://gigachat.devices.sberbank.ru/api/v1', 'verify_ssl_certs': False, 'main_model': 'GigaChat', 'temperature': 0.0} ---


--- GIGA CONFIG: {'base_url': 'https://gigachat.devices.sberbank.ru/api/v1', 'verify_ssl_certs': False, 'main_model': 'GigaChat', 'temperature': 0.0} ---


In [4]:
import os
import json
import uuid
import logging
import requests
import urllib3
import httpx
from dotenv import load_dotenv
from typing import List, Optional, Dict, Any, ClassVar, Type
from pydantic import BaseModel, Field

# Langchain imports
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_gigachat.embeddings.gigachat import GigaChatEmbeddings

# Agent Core imports
from sgr_deep_research.core.agent_definition import (
    AgentConfig, 
    LLMConfig, 
    PromptsConfig, 
    ExecutionConfig
)
from sgr_deep_research.core.agents.tool_calling_agent import ToolCallingAgent
from sgr_deep_research.core.tools import (
    BaseTool, 
    ClarificationTool,
    GeneratePlanTool,
    AdaptPlanTool,
    ReasoningTool,
    FinalAnswerTool
)
from sgr_deep_research.core.models import ResearchContext
from openai import AsyncOpenAI

# Настройка логирования
logging.basicConfig(level=logging.INFO)
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Загрузка переменных (.env)
load_dotenv(override=True)

# --- 1. Получение токена GigaChat ---
def get_gigachat_token(auth_key: str, scope: str = "GIGACHAT_API_PERS") -> str:
    """Получает токен доступа GigaChat с помощью ключа авторизации."""
    token_url = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"
    
    payload = {'scope': scope}
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Accept': 'application/json',
        'RqUID': str(uuid.uuid4()),
        'Authorization': f'Basic {auth_key}'
    }
    
    try:
        response = requests.post(token_url, headers=headers, data=payload, verify=False)
        response.raise_for_status()
        return response.json()['access_token']
    except Exception as e:
        print(f"Ошибка получения токена: {e}")
        if 'response' in locals():
             print(f"Ответ сервера: {response.text}")
        raise

# Получаем ключ из .env
auth_key = os.getenv("GIGACHAT_CREDENTIALS")
if not auth_key:
    raise ValueError("GIGACHAT_CREDENTIALS не найден в .env! Проверьте файл .env")

# Получаем актуальный токен
ACCESS_TOKEN = get_gigachat_token(auth_key)
print(f"Токен успешно получен: {ACCESS_TOKEN[:10]}...")

# Настройки для GigaChat (предполагаем наличие giga_settings как в оригинале, или задаем тут)
class GigaSettings:
    base_url = "https://gigachat.devices.sberbank.ru/api/v1"
    verify_ssl_certs = False
    main_model = "GigaChat-Pro"
    temperature = 0.0

giga_settings = GigaSettings()


# --- 2. Настройка FAISS (Каталог товаров и правила магазина) ---
embedding_function = GigaChatEmbeddings(
    base_url=giga_settings.base_url,
    access_token=ACCESS_TOKEN,
    verify_ssl_certs=giga_settings.verify_ssl_certs
)

documents = [
    Document(page_content="Конструктор 'Звездный крейсер' стоит 5000 рублей. Подходит для детей от 10 лет.", metadata={"id": "prod1"}),
    Document(page_content="Плюшевый медведь 'Мишутка' (50 см) стоит 1200 рублей.", metadata={"id": "prod2"}),
    Document(page_content="Настольная игра 'Монополия' стоит 2500 рублей.", metadata={"id": "prod3"}),
    Document(page_content="Доставка осуществляется бесплатно при заказе от 3000 рублей. Срок доставки 2-3 дня.", metadata={"id": "rule1"}),
    Document(page_content="Возврат товара возможен в течение 14 дней при сохранении упаковки.", metadata={"id": "rule2"})
]

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(documents)
vectorstore = FAISS.from_documents(splits, embedding_function)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})


# --- 3. Инструменты ---
class CatalogSearchTool(BaseTool):
    """Поиск информации о товарах и правилах магазина."""
    tool_name: ClassVar[str] = "search_catalog"
    description: ClassVar[str] = "Используй для поиска цен, описаний товаров и условий доставки/возврата."
    query: str = Field(..., description="Поисковый запрос (например: 'цена конструктора' или 'условия доставки')")

    async def __call__(self, context: ResearchContext) -> str:
        print(f"   [DEBUG] Поиск в каталоге: {self.query}")
        results = retriever.invoke(self.query)
        if results:
            found_info = "\n".join([f"- {doc.page_content}" for doc in results])
            return json.dumps(f"Найдено в базе:\n{found_info}", ensure_ascii=False)
        return json.dumps("В каталоге информации не найдено.", ensure_ascii=False)

class OrderCreationTool(BaseTool):
    """Оформление заказа на игрушки."""
    tool_name: ClassVar[str] = "create_order"
    description: ClassVar[str] = "Создание заказа. Возвращает номер заказа."
    items: str = Field(..., description="Список товаров и их количество")
    customer_info: str = Field(..., description="Имя и контактные данные покупателя")

    async def __call__(self, context: ResearchContext) -> str:
        print(f"   [DEBUG] Создание заказа: {self.items} для {self.customer_info}")
        return json.dumps({"status": "success", "order_id": "ORD-999", "message": "Order created successfully"}, ensure_ascii=False)

class OrderStatusTool(BaseTool):
    """Проверка статуса заказа по номеру."""
    tool_name: ClassVar[str] = "check_order_status"
    description: ClassVar[str] = "Проверка статуса заказа по ID (например, ORD-999)."
    order_id: str = Field(..., description="Номер заказа")

    async def __call__(self, context: ResearchContext) -> str:
        print(f"   [DEBUG] Проверка статуса заказа: {self.order_id}")
        if self.order_id == "ORD-999":
            return json.dumps("Статус: Передан в курьерскую службу. Ожидайте доставку завтра.", ensure_ascii=False)
        return json.dumps("Заказ с таким номером не найден.", ensure_ascii=False)


# --- 4. Конфигурация Агента ---
http_client = httpx.AsyncClient(verify=False)

llm_client = AsyncOpenAI(
    api_key=ACCESS_TOKEN,
    base_url=giga_settings.base_url,
    http_client=http_client
)

llm_config = LLMConfig(
    api_key=ACCESS_TOKEN,
    base_url=giga_settings.base_url,
    model=giga_settings.main_model,
    temperature=giga_settings.temperature,
)

prompts_config = PromptsConfig(
    system_prompt_str=(
        "Ты консультант магазина игрушек 'Детский Мир'. Отвечаешь на вопросы о товарах и помогаешь с заказами."
        "Список твоих инструментов = [CatalogSearchTool, OrderCreationTool, OrderStatusTool, ClarificationTool, GeneratePlanTool, AdaptPlanTool, ReasoningTool, FinalAnswerTool]"
        "Когда ответ готов - вызови FinalAnswerTool."
        "ПРАВИЛА: "
        "0. Будь вежлив и дружелюбен, используй эмодзи (🧸, 🚗)."
        "1. Сначала ищи информацию в каталоге (CatalogSearchTool), не выдумывай цены."
        "2. Если клиент хочет купить, сначала оформи заказ (OrderCreationTool), потом скажи номер заказа."
        "3. Если спрашивают про условия доставки, тоже смотри в CatalogSearchTool."
    ),
    initial_user_request_str=None,
    clarification_response_str="Уточните, пожалуйста, какую именно игрушку вы ищете?"
)

execution_config = ExecutionConfig(
    max_iterations=5,
    max_searches=0
)

tools_list = [
    CatalogSearchTool, OrderCreationTool, OrderStatusTool, 
    ClarificationTool, GeneratePlanTool, AdaptPlanTool, ReasoningTool, FinalAnswerTool
]


# --- 5. Запуск ---
async def run_agent_demo(user_input: str):
    print(f"\n>>> ПОКУПАТЕЛЬ: {user_input}")
    
    agent = ToolCallingAgent(
        task=user_input,
        openai_client=llm_client,
        llm_config=llm_config,
        prompts_config=prompts_config,
        execution_config=execution_config,
        toolkit=tools_list,
    )
    
    await agent.execute()
    
    print("AGENTS LOG:")
    for msg in agent.conversation:
        role = msg.get("role")
        content = msg.get("content")
        if role == "assistant" and content:
            print(f"[{role}]: {content}")
        elif role == "tool":
            print(f"[{role}]: {content}")

# Запуск
if __name__ == "__main__":
    # Сценарий 1: Вопрос по каталогу
    await run_agent_demo("Сколько стоит конструктор звездный крейсер и есть ли бесплатная доставка?")
    
    # Сценарий 2: Покупка
    await run_agent_demo("Хочу купить плюшевого медведя. Меня зовут Саша, телефон 89990000000. Оформите заказ.")

INFO:httpx:HTTP Request: POST https://gigachat.devices.sberbank.ru/api/v1/embeddings "HTTP/1.1 200 OK"


Токен успешно получен: eyJjdHkiOi...


INFO:sgr_deep_research.agents.tool_calling_agent_28a9634c-f00e-4ae4-8819-f67c87a19a4b:🚀 Starting for task: 'Сколько стоит конструктор звездный крейсер и есть ли бесплатная доставка?'
INFO:sgr_deep_research.agents.tool_calling_agent_28a9634c-f00e-4ae4-8819-f67c87a19a4b:Step 1 started



>>> ПОКУПАТЕЛЬ: Сколько стоит конструктор звездный крейсер и есть ли бесплатная доставка?


INFO:httpx:HTTP Request: POST https://gigachat.devices.sberbank.ru/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://gigachat.devices.sberbank.ru/api/v1/embeddings "HTTP/1.1 200 OK"
INFO:sgr_deep_research.agents.tool_calling_agent_28a9634c-f00e-4ae4-8819-f67c87a19a4b:
###############################################
🛠️ TOOL EXECUTION DEBUG:
    🔧 Tool Name: search_catalog
    📋 Tool Model: {
  "query": "конструктор звездный крейсер цена"
}
    🔍 Result: '"Найдено в базе:\n- Конструктор 'Звездный крейсер' стоит 5000 рублей. Подходит для детей от 10 лет.\n- Настольная игра 'Монополия' стоит 2500 рублей."...'
###############################################
INFO:sgr_deep_research.agents.tool_calling_agent_28a9634c-f00e-4ae4-8819-f67c87a19a4b:Step 2 started


...

AGENTS LOG:

>>> ПОКУПАТЕЛЬ: Хочу купить плюшевого медведя. Меня зовут Саша, телефон 89990000000. Оформите заказ.


INFO:httpx:HTTP Request: POST https://gigachat.devices.sberbank.ru/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:sgr_deep_research.agents.tool_calling_agent_d4bd554b-a05e-4f7c-a5e5-9fa224c42e30:
###############################################
🛠️ TOOL EXECUTION DEBUG:
    🔧 Tool Name: create_order
    📋 Tool Model: {
  "items": "плюшевый медведь 1",
  "customer_info": "Саша, 89990000000"
}
    🔍 Result: '{"status": "success", "order_id": "ORD-999", "message": "Order created successfully"}...'
###############################################
INFO:sgr_deep_research.agents.tool_calling_agent_d4bd554b-a05e-4f7c-a5e5-9fa224c42e30:Step 2 started


   [DEBUG] Создание заказа: плюшевый медведь 1 для Саша, 89990000000


INFO:httpx:HTTP Request: POST https://gigachat.devices.sberbank.ru/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:sgr_deep_research.agents.tool_calling_agent_d4bd554b-a05e-4f7c-a5e5-9fa224c42e30:
###############################################
🛠️ TOOL EXECUTION DEBUG:
    🔧 Tool Name: finalanswertool
    📋 Tool Model: {
  "reasoning": "Заказ на плюшевого медведя оформлен успешно и был присвоен номер заказа ORD-999. Ответ включает имя клиента и информацию о статусе заказа. Ответ позитивный и включает эмодзи для улучшения пользовательского опыта. 00000",
  "completed_steps": [
    "Оформлен заказ на плюшевого медведя на имя Саши с контактным телефоном 89990000000.00000"
  ],
  "answer": "Саша, Ваш заказ № ORD-999 на плюшевого медведя оформлен успешно. Пожалуйста, сохраняйте свой номер заказа. 🥰🧸",
  "status": "completed"
}
    🔍 Result: '{
  "reasoning": "Заказ на плюшевого медведя оформлен успешно и был присвоен номер заказа ORD-999. Ответ включает имя клиента и информацию о статусе зак

AGENTS LOG:


In [ ]:
giga_settings.verify_ssl_certs